In [ ]:
#作業目標:
#1. 藉由固定的 dataset, 來驗證不同loss function
#2. Dataset 的特性跟我們選用的loss function 對accrancy 的影響
#作業重點:
#請分別選用 "MSE", "binary _crossentropy"
#查看Train/test accurancy and loss rate

In [1]:
from keras.datasets import cifar10
import numpy as np
np.random.seed(10)

Using TensorFlow backend.


In [2]:
#取得Keras Dataset  並分成Training 與 Test set
(x_img_train,y_label_train),(x_img_test,y_label_test)=cifar10.load_data()

In [3]:
#確認 CIFAR10 Dataset 資料維度
print("train data:",'images:',x_img_train.shape,
      " labels:",y_label_train.shape) 
print("test  data:",'images:',x_img_test.shape ,
      " labels:",y_label_test.shape) #shape它的功能是查看矩阵或者数组的维数

train data: images: (50000, 32, 32, 3)  labels: (50000, 1)
test  data: images: (10000, 32, 32, 3)  labels: (10000, 1)


In [4]:
#資料正規化
x_img_train_normalize = x_img_train.astype('float32') / 255.0 #数据类型转换astype
x_img_test_normalize = x_img_test.astype('float32') / 255.0

In [5]:
#針對Label 做 ONE HOT ENCODE
from keras.utils import np_utils
y_label_train_OneHot = np_utils.to_categorical(y_label_train)
y_label_test_OneHot = np_utils.to_categorical(y_label_test)
y_label_test_OneHot.shape
#// np_utils.to_categorical將 training 的 label 進行 one-hot encoding

(10000, 10)

In [6]:
#建立模型
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D

In [7]:
model = Sequential() # 宣告採用序列模型

In [8]:
#卷積層1
model.add(Conv2D(filters=32,kernel_size=(3,3),
                 input_shape=(32, 32,3), 
                 activation='relu',   ##model.add(Activation('relu)) # 激活函數是relu
                 padding='same'))
#卷積層(Convolution Layer)參數
#二維(Conv2D)
#Filters：濾波器數目
#ernel_size：卷積核大小
#padding：補零方式
#一律補零(same)

Instructions for updating:
Colocations handled automatically by placer.


In [9]:

model.add(Dropout(rate=0.25))
#dropout是指在深度学习网络的训练过程中，对于神经网络单元，按照一定的概率将其暂时从网络中丢弃。注意是暂时，对于随机梯度下降来说，由于是随机丢弃，故而每一个mini-batch都在训练不同的网络。
#rate 利率

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [10]:
model.add(MaxPooling2D(pool_size=(2, 2)))#池化層 二維(MaxPooling2D)
#pool_size：整数或长为2的整数tuple，代表在两个方向（竖直，水平）上的下采样因子

In [11]:
model.add(Conv2D(filters=64, kernel_size=(3, 3), 
                 activation='relu', padding='same'))
##Filters：濾波器數目
#kernel_size：卷積核大小
#padding：補零方式
#一律補零(same)

In [12]:
model.add(Dropout(0.25))

In [13]:
model.add(MaxPooling2D(pool_size=(2, 2)))

In [14]:
#建立神經網路(平坦層、隱藏層、輸出層)
model.add(Flatten())
model.add(Dropout(rate=0.25))

In [15]:
model.add(Dense(1024, activation='relu'))
model.add(Dropout(rate=0.25))
#Dense層屬於網絡層- (1024 是輸出的維度

In [16]:
model.add(Dense(10, activation='softmax'))

In [17]:
#檢查model 的STACK
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
dropout_2 (Dropout)          (None, 16, 16, 64)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4096)              0         
__________

In [18]:
try:
    model.load_weights("SaveModel/cifarCnnModel.h5")
    print("載入模型成功!繼續訓練模型")
except :    
    print("載入模型失敗!開始訓練一個新模型")

載入模型失敗!開始訓練一個新模型


In [19]:
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])


In [20]:
train_history=model.fit(x_img_train_normalize, y_label_train_OneHot,
                        validation_split=0.25,
                        epochs=12, batch_size=128, verbose=1)    

Instructions for updating:
Use tf.cast instead.
Train on 37500 samples, validate on 12500 samples
Epoch 1/12
37500/37500 [==============================] - 186s 5ms/step - loss: 2.1650 - acc: 0.1982 - val_loss: 2.0515 - val_acc: 0.2962
Epoch 2/12
37500/37500 [==============================] - 193s 5ms/step - loss: 1.9549 - acc: 0.2956 - val_loss: 1.9352 - val_acc: 0.3459
Epoch 3/12
37500/37500 [==============================] - 179s 5ms/step - loss: 1.8338 - acc: 0.3473 - val_loss: 1.8416 - val_acc: 0.3825
Epoch 4/12
37500/37500 [==============================] - 178s 5ms/step - loss: 1.7303 - acc: 0.3821 - val_loss: 1.7484 - val_acc: 0.4174
Epoch 5/12
37500/37500 [==============================] - 178s 5ms/step - loss: 1.6497 - acc: 0.4101 - val_loss: 1.6803 - val_acc: 0.4393
Epoch 6/12
37500/37500 [==============================] - 178s 5ms/step - loss: 1.5856 - acc: 0.4313 - val_loss: 1.6506 - val_acc: 0.4509
Epoch 7/12
37500/37500 [==============================] - 178s 5ms/step - 